# Librairies

In [1]:
import os
import json

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.utils import Sequence, to_categorical

print(tf.test.is_gpu_available())

True


# Data

In [2]:
list_IDs = {}
with open("data/dataset.json", "r") as file:
    list_IDs = json.load(file)

In [4]:
class NTUSequence(Sequence):
    
    def __init__(self, list_IDs, path, batch_size=32, max_frame=1, one_hot=True):
        self.x = list_IDs
        self.y = labels
        self.batch_size = batch_size
        self.max_frame = max_frame
        self.path = path
        if one_hot:
            self.one_hot_encode()

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = np.array([self.y[ID] for ID in batch_x])
        
        X = [np.load(self.path + ID + '.npy')[:, :, :, np.newaxis] for ID in batch_x]
        
        if not(self.max_frame is None):
            X = np.stack(
                [x[:max_frame] if x.shape[0] >= max_frame else np.pad(x, ((0, max_frame-x.shape[0]),(0,0),(0,0),(0,0)), "constant") for x in X]
            )     
        
        return X, batch_y

    def one_hot_encode(self):
        labels = [val for val in self.y.values()]
        ids = [key for key in self.y.keys()]
        labels = to_categorical(labels)
        self.y = {ID: label for ID, label in zip(ids, labels)}

In [5]:
trainset = NTUSequence(list_IDs["train"], path="D:/PFE/train/")
testset = NTUSequence(list_IDs["validation"], path="D:/PFE/test/")

TypeError: __init__() missing 1 required positional argument: 'labels'

# Architecture

In [ ]:
convnet = models.Sequential([
    
    layers.Conv2D(64, 3, activation="relu", padding="same", input_shape=(25, 25, 1)),
    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
                        
    layers.Conv2D(512, 3, activation="relu", padding="same"),
    layers.Conv2D(512, 3, activation="relu", padding="same"),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten()
                        ])

In [ ]:
convnet.summary()

In [ ]:
predictor = models.Sequential([
    layers.Input((None, 25, 25, 1)),
    layers.TimeDistributed(convnet),
    
    layers.LSTM(9, activation="relu"),
    layers.Reshape((3,3,1))
    
    layers.Conv2DTranspose(1, 3, strides=2, activation="relu", padding="same")
    layers.Conv2DTranspose(1, 3, strides=2, activation="relu", padding="same")
    layers.Conv2DTranspose(1, 3, strides=2, activation="relu")
    
    layers.Reshape((25,25))
])

In [ ]:
predictor.summary()